# Run custom versions of the FastScape landscape evolution model

Let's import some packages first (you can install them using ``conda``).

In [1]:
import numpy as np
import xarray as xr
import xsimlab as xs
import fastscape

In [2]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

xarray-simlab version:  0.4.1
fastscape version:  0.1.0beta


## Create a new component (process)

Sometimes there is no existing module for the simulation that we'd like to run. The ``xarray-simlab`` framework allows to create new components and plug them to existing models with very little effort.

Here is an example below that chooses randomly spatially variable values for the $K$ parameter of the stream-power law.

In [3]:
from fastscape.processes import StreamPowerChannel, RasterGrid2D

A ``xarray-simlab`` component (or process) is a (most-often) small, "classic" Python class that is decorated by ``xsimlab.process``.

More info on how to create processes: https://xarray-simlab.readthedocs.io/en/latest/create_model.html

In [4]:
@xs.process
class SPLRandomK:
    grid_shape = xs.foreign(RasterGrid2D, 'shape')
    k_coef = xs.foreign(StreamPowerChannel, 'k_coef', intent='out')
    
    def initialize(self):
        self.k_coef = np.random.uniform(1e-5, 1e-4, size=self.grid_shape)

## Update the basic model with this new component

Let's first import ``basic_model``

In [5]:
from fastscape.models import basic_model

basic_model

<xsimlab.Model (16 processes, 8 inputs)>
grid
    length          [in] ('shape_yx',) total grid length in (y, x)
    shape           [in] ('shape_yx',) nb. of grid nodes in (y, x)
fs_context
boundary
    status          [in] () or ('border',) node status at borders
uplift
    rate            [in] () or ('y', 'x') uplift rate
tectonics
init_topography
surf2erode
diffusion
    diffusivity     [in] () or ('y', 'x') diffusivity (transport co...
init_erosion
flow
drainage
spl
    k_coef          [in] () or ('y', 'x') bedrock channel incision ...
    slope_exp       [in] slope exponent
    area_exp        [in] drainage area exponent
erosion
vmotion
topography
terrain

Then we just need ``update_processes``: 

In [6]:
new_model = basic_model.update_processes({'randk': SPLRandomK})

new_model

<xsimlab.Model (17 processes, 7 inputs)>
grid
    length          [in] ('shape_yx',) total grid length in (y, x)
    shape           [in] ('shape_yx',) nb. of grid nodes in (y, x)
fs_context
boundary
    status          [in] () or ('border',) node status at borders
uplift
    rate            [in] () or ('y', 'x') uplift rate
tectonics
init_topography
surf2erode
diffusion
    diffusivity     [in] () or ('y', 'x') diffusivity (transport co...
init_erosion
flow
drainage
randk
spl
    slope_exp       [in] slope exponent
    area_exp        [in] drainage area exponent
erosion
vmotion
topography
terrain

## Run the new model (reuse an existing setup)

Let's import below the setup that we have created in the ``run_basic_model`` notebook:

In [7]:
in_ds = xr.load_dataset('basic_input.nc')

Let's update the setup and run the new model created above:

In [8]:
with new_model:
    out_ds = (
        in_ds.xsimlab.filter_vars()
             .xsimlab.update_vars(output_vars={'randk__k_coef': None})
             .xsimlab.run()
    )


out_ds

<xarray.Dataset>
Dimensions:                 (border: 4, out: 11, shape_yx: 2, time: 101, x: 201, y: 101)
Coordinates:
  * border                  (border) <U6 'left' 'right' 'top' 'bottom'
  * out                     (out) float64 0.0 1e+05 2e+05 ... 8e+05 9e+05 1e+06
  * time                    (time) float64 0.0 1e+04 2e+04 ... 9.9e+05 1e+06
  * x                       (x) float64 0.0 100.0 200.0 ... 1.99e+04 2e+04
  * y                       (y) float64 0.0 100.0 200.0 ... 9.9e+03 1e+04
Dimensions without coordinates: shape_yx
Data variables:
    boundary__status        (border) object 'looped' 'looped' ... 'fixed_value'
    diffusion__diffusivity  float64 0.1
    drainage__area          (out, y, x) float64 2e+04 6e+04 ... 2e+04 2e+04
    flow__basin             (out, y, x) int64 16834 5990 18174 ... 14828 1553
    grid__length            (shape_yx) float64 1e+04 2e+04
    grid__shape             (shape_yx) int64 101 201
    randk__k_coef           (y, x) float64 4.242e-05 7.11e-05

Let's look at the topography:

In [9]:
import hvplot.xarray
import matplotlib.pyplot as plt

out_ds.topography__elevation.hvplot.image(x='x', y='y',
                                          cmap=plt.cm.viridis,
                                          groupby='out')

:DynamicMap   [out]
   :Image   [x,y]   (topography__elevation)

## Exercices

- Update the component so that the user can provide a range for the randomly generated parameter values
- Update the component so that new random values are generated at each time step
- Any idea scientifically more interesting?